In [2]:
from __future__ import print_function, unicode_literals
from unicodedata import normalize
import library_py2 as lib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.collocations import *
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, Masking
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import argparse
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

In [4]:
data_path='/models'

In [13]:
df = pd.read_pickle('preprocessed_data2.pkl')

In [5]:
df2 = pd.read_pickle('lstm_ans2.pkl')

In [14]:
len(df)

130319

In [15]:
df.head()

,context,question,answers,plausible_answers,is_impossible,context_lemma_pos,question_lemma_pos,answers_lemma_pos,plausible_answers_lemma_pos,context_lemma_pos_tfidf,question_lemma_pos_tfidf
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (start, NN), (bec...","[(late, RB), (1990s, NNS)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(when, WRB, 0.2), (beyonce, NN, 0.2), (start,..."
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(what, WP), (area, NNS), (beyonce, VBP), (com...","[(sing, VBG), (dance, VBG)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(what, WP, 0.2), (area, NNS, 0.2), (beyonce, ..."
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(when, WRB, 0.125), (beyonce, NN, 0.125), (le..."
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(in, IN), (city, NN), (state, NN), (beyonce, ...","[(houston, NN), (texas, NN)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(in, IN, 0.2), (city, NN, 0.2), (state, NN, 0..."
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(in, IN), (decade, NN), (beyonce, NN), (becom...","[(late, RB), (1990s, NNS)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(in, IN, 0.2), (decade, NN, 0.2), (beyonce, N..."


In [16]:
df_ans = df.loc[df.is_impossible==0]

In [18]:
len(df_ans)

86821

In [25]:
df_ans['answer_len'] = df_ans.answers_lemma_pos.apply(lambda x: len(x))

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
df_ans_1 = df_ans.loc[df_ans['answer_len']==1]

In [34]:
len(df_ans_1)

31989

In [41]:
df_ans_1.head()

,context,question,answers,plausible_answers,is_impossible,context_lemma_pos,question_lemma_pos,answers_lemma_pos,plausible_answers_lemma_pos,context_lemma_pos_tfidf,question_lemma_pos_tfidf,answer_len
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyonce, NN), (leave, VBP), (de...","[(2003, CD)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(when, WRB, 0.125), (beyonce, NN, 0.125), (le...",1
11,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé release Dangerously in Love?,2003,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(when, WRB), (beyoncé, NN), (release, NN), (d...","[(2003, CD)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(when, WRB, 0.2), (beyoncé, NN, 0.2), (releas...",1
12,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,How many Grammy awards did Beyoncé win for her...,five,,0,"[(beyoncé, NN), (giselle, NN), (knowlescarter,...","[(how, WRB), (many, JJ), (grammy, JJ), (award,...","[(five, CD)]",,"[(beyoncé, NN, 0.013333333333333334), (giselle...","[(how, WRB, 0.1111111111111111), (many, JJ, 0....",1
15,Following the disbandment of Destiny's Child i...,"After her second solo album, what other entert...",acting,,0,"[(follow, VBG), (disbandment, NN), (destiny, N...","[(after, IN), (second, JJ), (solo, NN), (album...","[(act, VBG)]",,"[(follow, VBG, 0.008928571428571428), (disband...","[(after, IN, 0.125), (second, JJ, 0.125), (sol...",1
17,Following the disbandment of Destiny's Child i...,"To set the record for Grammys, how many did Be...",six,,0,"[(follow, VBG), (disbandment, NN), (destiny, N...","[(to, TO), (set, VB), (record, NN), (grammys, ...","[(six, CD)]",,"[(follow, VBG, 0.008928571428571428), (disband...","[(to, TO, 0.14285714285714285), (set, VB, 0.14...",1


In [42]:
df_ans_1_pos = df_ans_1.drop(columns=['context', 'question', 'answers', 'plausible_answers', 'is_impossible', 'plausible_answers_lemma_pos','context_lemma_pos_tfidf','question_lemma_pos_tfidf'])

In [44]:
df_ans_1_pos.to_pickle('df_pos_.pkl')

In [10]:
len(ndf)

10000

In [13]:
ndf.head()

,answers_lstm,text,text_len,answers_lstm_len,fulltext
0,ebola,one way prevent slow transmission infectious d...,116,1,one way prevent slow transmission infectious d...
1,coup,early year popper impress marxism whether comm...,110,1,early year popper impress marxism whether comm...
2,1980s,us uk france italy majority western country co...,84,1,us uk france italy majority western country co...
3,8th,vocalization marker indicate specific vowel so...,146,1,vocalization marker indicate specific vowel so...
4,hun,engagement without controversy philip financia...,72,1,engagement without controversy philip financia...


In [ ]:
glove50 = pd.read_txt

In [16]:
vocabulary = lib.get_total_vocab(ndf)

In [48]:
coefs_sum=np.zeros((50,))
coefs_sum.shape

(50,)

In [50]:
dim=50
embeddings_index = {}
f = open(os.path.join('glove.6B/glove.6B.{}d.txt'.format(dim)))
coefs_sum=np.zeros((dim,))
coefs_count = 0
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    coefs_sum += coefs
    coefs_count+=1
f.close()
coefs_mean = coefs_sum/coefs_count

In [52]:
coefs_mean = coefs_sum/coefs_count

In [53]:
coefs_mean

array([-0.12920061, -0.28866239, -0.01224894, -0.05676689, -0.20211109,
       -0.08389026,  0.33359737,  0.16045146,  0.03867495,  0.17833092,
        0.0469662 , -0.00285779,  0.29099851,  0.04613723, -0.20923842,
       -0.066131  , -0.06822448,  0.07665885,  0.31339918,  0.17848512,
       -0.12257719, -0.09916928, -0.07495973,  0.06413206,  0.14441256,
        0.608946  ,  0.17463101,  0.05335403, -0.01273826,  0.03474108,
       -0.81239567, -0.04688727,  0.20193533,  0.20311115, -0.03935654,
        0.06967518, -0.01553655, -0.03405275, -0.06528025,  0.12250092,
        0.13992005, -0.17446305, -0.08011841,  0.08495219, -0.01041645,
       -0.13704901,  0.20127088,  0.10069294,  0.00653007,  0.0168515 ])

In [24]:
print(values[0])

sandberger


In [49]:
embeddings_index['word'].shape

(50,)

In [54]:
dim=50
embeddings_index = {}
f = open(os.path.join('glove.6B/glove.6B.{}d.txt'.format(dim)))
coefs_sum=np.zeros((dim,))
coefs_count = 0
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    coefs_sum += coefs
    coefs_count+=1
f.close()
coefs_mean = coefs_sum/coefs_count

test = lib.file_to_word_ids(vocabulary, ndf[:100], input_col='text', output_col = 'answers_lstm', 
                            max_input=ndf.text_len.max(), max_output=ndf.answers_lstm_len.max())

In [55]:
reversed_dictionary = dict(zip(vocabulary.values(), vocabulary.keys()))

In [ ]:
reversed_dictionary[230]

In [59]:
embedding_matrix = np.zeros((len(vocabulary) + 1, dim))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = coefs_mean

In [63]:
coefs_mean

array([-0.12920061, -0.28866239, -0.01224894, -0.05676689, -0.20211109,
       -0.08389026,  0.33359737,  0.16045146,  0.03867495,  0.17833092,
        0.0469662 , -0.00285779,  0.29099851,  0.04613723, -0.20923842,
       -0.066131  , -0.06822448,  0.07665885,  0.31339918,  0.17848512,
       -0.12257719, -0.09916928, -0.07495973,  0.06413206,  0.14441256,
        0.608946  ,  0.17463101,  0.05335403, -0.01273826,  0.03474108,
       -0.81239567, -0.04688727,  0.20193533,  0.20311115, -0.03935654,
        0.06967518, -0.01553655, -0.03405275, -0.06528025,  0.12250092,
        0.13992005, -0.17446305, -0.08011841,  0.08495219, -0.01041645,
       -0.13704901,  0.20127088,  0.10069294,  0.00653007,  0.0168515 ])

In [ ]:
# here will implement a broadcasting multiplication to the 

In [61]:
embedding_matrix[230]

array([ 0.42331001,  0.60319   , -0.37059   , -0.29663   , -1.29349995,
        0.25779   , -0.10106   , -0.36515999,  1.05229998, -0.087572  ,
       -0.021456  ,  0.37215   , -0.58888   ,  0.68932003, -0.77908999,
       -0.84881002,  0.68708998,  0.10415   ,  0.95573997,  0.49412999,
        0.54812998,  0.049937  , -1.10389996,  0.36725   ,  0.36078   ,
        0.9059    ,  0.64317   , -0.057156  , -0.50313997, -0.36008   ,
       -0.58819997,  1.33729994,  0.33414   ,  1.49109995, -0.17323001,
        0.99133003, -0.33748001, -0.02083   , -1.03939998,  0.86097997,
        0.19337   , -0.31277001,  0.39899001, -0.010284  , -0.35313001,
       -0.88028997,  0.46779001,  0.83437002,  0.25479999, -1.97049999])

In [ ]:
w = 'lst_mm'
if w contains

In [103]:
def load_data(data, use_glove=True, embedding_dim=50):
    # 
    data.reset_index(inplace=True, drop=True)
    
    train_df = data.iloc[:round(0.8*len(data))]
    valid_df = data.iloc[round(0.8*len(data)):round(0.9*len(data))]
    test_df = data.iloc[round(0.9*len(data)):]
    

    # build the complete vocabulary, then convert text data to dict of integer-word pairs
    vocabulary = lib.get_total_vocab(data, column='text')
    train_data = lib.file_to_word_ids(vocabulary, train_df, input_col='text', output_col = 'answers_lstm', 
                                      max_input=data.text_len.max(), max_output=data.answers_lstm_len.max())
    valid_data = lib.file_to_word_ids(vocabulary, valid_df, input_col='text', output_col = 'answers_lstm', 
                                      max_input=data.text_len.max(), max_output=data.answers_lstm_len.max())
    test_data = lib.file_to_word_ids(vocabulary, test_df, input_col='text', output_col = 'answers_lstm', 
                                      max_input=data.text_len.max(), max_output=data.answers_lstm_len.max())
    vocabulary_size = len(vocabulary)
    reversed_dictionary = dict(zip(vocabulary.values(), vocabulary.keys()))
    
    if use_glove:
        dim=embedding_dim
        embeddings_index = {}
        f = open(os.path.join('glove.6B/glove.6B.{}d.txt'.format(dim)))
        coefs_sum=np.zeros((dim,))
        coefs_count = 0
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
            coefs_sum += coefs
            coefs_count+=1
        f.close()
        coefs_mean = coefs_sum/coefs_count
        
        embedding_matrix = np.zeros((len(vocabulary), dim))
        for word, i in vocabulary.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
            else:
                embedding_matrix[i] = coefs_mean

# test = lib.file_to_word_ids(vocabulary, ndf[:100], input_col='text', output_col = 'answers_lstm', 
#                             max_input=ndf.text_len.max(), max_output=ndf.answers_lstm_len.max())
        
        
    
    print("First five train ... ", train_data[:5])
    print("Vocabulary   ", vocabulary)
    print(" Size of vocab .  ", vocabulary_size)
    train_data_lst=train_data.tolist()
    print(" ".join([reversed_dictionary[x] for x in train_data_lst[10]]))
    return train_data, valid_data, test_data, vocabulary, reversed_dictionary, vocabulary_size, embedding_matrix



In [104]:
train_data, valid_data, test_data, vocabulary, reversed_dictionary, vocabulary_size, embedding_matrix, embedding_dim = load_data(ndf)


First five train ...  [[    0     0     0 ...  9151     2  6412]
 [    0     0     0 ...  5156     2  6620]
 [    0     0     0 ...   808     2 45112]
 [    0     0     0 ...  4991     2 27428]
 [    0     0     0 ... 27769     2 35636]]
Vocabulary    {'cm': 1, '': 2, 'elwell': 3, 'donnelly': 4, 'intercessor': 5, 'aef': 6, 'waiver': 7, 'vocationally': 8, 'moyes': 9, 'aflaf2': 10, 'encompasseth': 11, 'ocelli': 12, 'portable': 13, 'socialist': 14, 'hub—built': 15, 'energized': 16, 'masquerader': 17, 'paratrooper': 18, 'methionine': 19, 'maritain': 20, 'sandberg': 21, 'planner': 22, 'съел': 23, 'tac': 24, 'reinterpretation': 25, 'coelho': 26, 'gesamtschule': 27, 'borton': 28, '8–12': 29, 'backpayments': 30, 'avignon': 31, 'microsoft': 32, 'oathtaking': 33, '1783': 34, 'fo': 35, '1442': 36, 'invercargill': 37, 'event': 38, 'contagiously': 39, 'biographic': 40, 'incorporated': 41, 'udabhandapura': 42, 'disarm': 43, 'vaiśeṣika': 44, 'touchcitation': 45, 'κάλπικη': 46, 'pinnacle': 47, 'brasil

In [105]:
vocabulary_size

51236

In [106]:
embedding_matrix.shape

(51236, 50)

In [107]:
embedding_dim=50

In [108]:
class KerasBatchGenerator(object):
    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step, answer_length=1, use_glove=True):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        self.current_idx = 0
        self.skip_step = skip_step
        self.vocabulary_size = len(vocabulary)
        self.answer_length = answer_length
        self.use_glove = use_glove
        
        
    #     num_steps = nr of words that will be fed into the input layer; i.e. the nr of words the model will use to predict the next word
    #     skip_steps = how far to move the window after the prediction is made

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps)) # input nn nodes, same dimensions as batchsize x nr of words in one "window"
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary_size)) # output nodes
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data): # once the ticker goes over the length of data, reset it
                    self.current_idx = 0
                x[i,:] = self.data[self.current_idx][:-self.answer_length]
                
#                 x[i,:] = self.data[j][:-2]
                # input is the segment of data from current index to current index + step-increment
#                 temp_y = self.data[self.current_idx+1 : self.num_steps + self.current_idx + 1]
                temp_y = self.data[self.current_idx][self.answer_length:]
                # converts all the output y into a 1-hot representation
                y[i,:, :] = to_categorical(temp_y, num_classes=self.vocabulary_size)
#                 self.current_idx += self.skip_step
                self.current_idx+=1
            
            
            yield x, y
        
            
        # x dimensions: nr of samples we specify in the batch first, number of w to base predictions on 
        # y has 3 dims: batchsize, nr time steps and also size of vocabulary


In [109]:
ndf.text_len.max()

313

In [110]:
num_steps = ndf.text_len.max()
batch_size = 10


In [111]:
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps+1)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps+1)

In [112]:
hidden_size = embedding_dim
use_dropout=True
num_epochs=20
hidden_size

50

In [113]:
hidden_size = embedding_dim
use_dropout=True
num_epochs=20
hidden_size

model_glove_50d = Sequential([
    layers.Embedding(input_dim=vocabulary_size, 
                     output_dim=hidden_size,input_length=num_steps,
                     weights = [embedding_matrix],
                     mask_zero=True, trainable=False),
    layers.LSTM(hidden_size, return_sequences=True),
    layers.LSTM(hidden_size, return_sequences=True),
    layers.LSTM(hidden_size, dropout=0.25, return_sequences=True),
    layers.TimeDistributed(Dense(vocabulary_size)),
    layers.Activation('softmax')
])


optimizer=Adam()
model_glove_50d.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model_glove_50d.summary())

In [114]:

optimizer=Adam()
model_glove_50d.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model_glove_50d.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 313, 50)           2561800   
_________________________________________________________________
lstm_10 (LSTM)               (None, 313, 50)           20200     
_________________________________________________________________
lstm_11 (LSTM)               (None, 313, 50)           20200     
_________________________________________________________________
lstm_12 (LSTM)               (None, 313, 50)           20200     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 313, 51236)        2613036   
_________________________________________________________________
activation_4 (Activation)    (None, 313, 51236)        0         
Total params: 5,235,436
Trainable params: 2,673,636
Non-trainable params: 2,561,800
__________________________________________________________

In [50]:
s = [0.8288, 0.9812, 0.12]
np.asarray(s)

array([0.8288, 0.9812, 0.12  ])

In [115]:
model_glove_50d.fit_generator(train_data_generator.generate(), steps_per_epoch = len(train_data)//(batch_size*20), 
                    epochs=num_epochs,
                    validation_data=valid_data_generator.generate(),
                    validation_steps=len(valid_data)//(batch_size))

Epoch 1/20
40/40 [==============================] - 2440s 61s/step - loss: 10.3585 - categorical_accuracy: 0.0356 - val_loss: 9.2125 - val_categorical_accuracy: 0.0258
Epoch 2/20
40/40 [==============================] - 2361s 59s/step - loss: 8.6786 - categorical_accuracy: 0.0255 - val_loss: 8.1675 - val_categorical_accuracy: 0.0260
Epoch 3/20
40/40 [==============================] - 2379s 59s/step - loss: 8.0350 - categorical_accuracy: 0.1166 - val_loss: 7.7222 - val_categorical_accuracy: 0.1985
Epoch 4/20
40/40 [==============================] - 2352s 59s/step - loss: 7.7615 - categorical_accuracy: 0.1901 - val_loss: 7.6589 - val_categorical_accuracy: 0.1962
Epoch 5/20
40/40 [==============================] - 2368s 59s/step - loss: 7.9010 - categorical_accuracy: 0.1668 - val_loss: 7.6359 - val_categorical_accuracy: 0.1978
Epoch 6/20
40/40 [==============================] - 2363s 59s/step - loss: 7.9572 - categorical_accuracy: 0.1633 - val_loss: 7.6474 - val_categorical_accuracy: 0.19

In [9]:
pd.read_pickle('lstm_1.pkl')

,answers_lstm,text,text_len,answers_lstm_len,fulltext
0,late 1990s,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
1,sing dance,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
2,2003,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,83,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
3,houston texas,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
4,late 1990s,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
...,...,...,...,...,...
86815,turkish airline,main international airport serve kathmandu thu...,106,3,main international airport serve kathmandu thu...
86816,oregon,kathmandu metropolitan city kmc order promote ...,80,2,kathmandu metropolitan city kmc order promote ...
86817,rangoon,kathmandu metropolitan city kmc order promote ...,76,2,kathmandu metropolitan city kmc order promote ...
86818,minsk,kathmandu metropolitan city kmc order promote ...,77,2,kathmandu metropolitan city kmc order promote ...


In [405]:
train_data[:,:-1]

array([[    0,     0,     0, ..., 45047,  9151,     2],
       [    0,     0,     0, ...,  1477,  5156,     2],
       [    0,     0,     0, ..., 26796,   808,     2],
       ...,
       [    0,     0,     0, ..., 23936, 26261,     2],
       [    0,     0,     0, ..., 12794, 31568,     2],
       [    0,     0,     0, ..., 20415,  5706,     2]], dtype=int32)

In [8]:
hidden_size = embedding_dim
use_dropout=True
num_epochs=20
hidden_size
num_steps=df1.text_len.max()

model_glove_test = Sequential([
    layers.Embedding(input_dim=vocabulary_size, 
                     output_dim=hidden_size,input_length=num_steps,
                     weights = [embedding_matrix],
                     mask_zero=True, trainable=False),
    layers.LSTM(hidden_size, return_sequences=True),
    layers.LSTM(hidden_size, dropout=0.25, return_sequences=True),
    layers.TimeDistributed(Dense(vocabulary_size)),
    layers.Activation('softmax')
])


optimizer=Adam()
model_glove_test.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model_glove_test.summary())

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:560: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 313, 50)           2561800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 313, 50)           20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 313, 50)           20200     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 313, 51236)        2613036   
_________________________________________________________________
activation_1 (Activation)    (None, 313, 51236)        0         
Total params: 5,215,236
Trainable params: 2,653,436
Non-trainable params: 2,561,800
_________________________________________________________________
None


In [422]:

np.random.random()

0.09186436435720047

In [429]:
train_data[:,-1:]

array([[ 6412],
       [ 6620],
       [45112],
       ...,
       [43766],
       [27221],
       [32924]], dtype=int32)

In [5]:
df1.head()

,answers_lstm,text,text_len,answers_lstm_len,fulltext
0,arabic,lebanon part christian population considers le...,50,1,lebanon part christian population considers le...
1,1668,chamorros guams indigenous people settle islan...,72,1,chamorros guams indigenous people settle islan...
2,sarong,dress westernize clothing jean tshirts somali ...,30,1,dress westernize clothing jean tshirts somali ...
3,archiveitorg,1996 brewster kahle bruce gilliat develop soft...,87,1,1996 brewster kahle bruce gilliat develop soft...
4,mahayana,accord jan natty term mahāyāna great vehicle o...,36,1,accord jan natty term mahāyāna great vehicle o...


In [6]:
train_data, valid_data, test_data, vocabulary, reversed_dictionary, vocabulary_size, embedding_matrix, embedding_dim = lib.load_data(df1)


First five train sentences in vectorized format :  [[    0     0     0 ... 46848  5210 29633]
 [    0     0     0 ... 36824 22191 21157]
 [    0     0     0 ... 32780 47241 25563]
 [    0     0     0 ... 16527 47774  6487]
 [    0     0     0 ... 49414 32127 20062]]
Vocabulary examples :    cosmological -  17742 conservatoire -  36720 bring -  32949
 Size of vocabulary :  51236
sapwood alburnum young outermost wood grow tree live wood principal function conduct water root leaf store give back accord season reserve prepare leaf however time become competent conduct water xylem tracheid vessel lose cytoplasm cell therefore functionally dead wood tree first form sapwood leave tree bear vigorous growth large volume sapwood require hence tree make rapid growth open thicker sapwood size tree specie grow dense forest sometimes tree specie form heartwood grow open may become considerable size 30 cm diameter heartwood begin form example secondgrowth hickory opengrown pine term interchangeable s

In [460]:
(train_data[0,:])

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [465]:
train_data[4,1:]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [ ]:
train_x = np.zeros((len(train_data), 313)) # input nn nodes, sam
train_y = np.zeros((len(train_data), 313, vocabulary_size))

for i in range(len(train_data)):
    train_x[i,] = train_data[i,:-1]
    temp_y = train_data[i,1:]
    train_y[i, :, :] = to_categorical(temp_y, num_classes=vocabulary_size)


In [1]:
train_y.shape

NameError: name 'train_y' is not defined

In [462]:
model_glove_test.fit(train_x,train_y, steps_per_epoch = len(train_data)//(batch_size*20), 
                    epochs=num_epochs)

ValueError: Error when checking target: expected activation_5 to have shape (313, 51236) but got array with shape (1, 51236)

In [117]:
model_glove_50d.save('models/model_glove_50d_dropout25.h5')

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary_size, hidden_size, input_length=num_steps, mask_zero=True))
model.add((Masking(mask_value=0)))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary_size)))
model.add(Activation('softmax'))

In [ ]:

optimizer=Adam()
model.compile(loss='categorical_hinge', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())



In [ ]:
model.fit_generator(train_data_generator.generate(), steps_per_epoch = len(train_data)//(batch_size*20), 
                    epochs=num_epochs,
                    validation_data=valid_data_generator.generate(),
                    validation_steps=len(valid_data)//(batch_size))

In [122]:
model_glove_50d = load_model('models/model_glove_50d_dropout25.h5')

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:560: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/engine/saving.py:251: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if weight_names:


In [123]:
model_glove_50d

In [120]:
vocabulary_size

51236

In [124]:
model_glove_50d.input

<tf.Tensor 'embedding_4_input_1:0' shape=(?, 313) dtype=float32>

In [126]:
model_glove_50d.output

<tf.Tensor 'activation_4_1/truediv:0' shape=(?, 313, 51236) dtype=float32>

In [136]:
d = np.zeros((1,num_steps))
d[0,:] = train_data[101][:-1]


In [157]:
seq=''
for x in d[0]:
    if x!=0:
        seq += reversed_dictionary[x] + "  "
seq += '\n' + "ANSWER:  " + reversed_dictionary[train_data[101][-1]] + "; "
pred  = model_glove_50d.predict(d)
predicted_word = np.argmax(pred[0, num_steps-1, :])  # test whether it needs to be -1, given that we're outputting the final word (the num_steps is equal to the max length of text)
seq += "PREDICTION:  " 
for j in range(1,5):
    seq += reversed_dictionary[predicted_word] + "  ;;;  "
    
print(seq)

first  mention  name  rajasthan  appear  james  tod  1829  publication  annals  antiquity  rajasthan  central  western  rajpoot  state  india  early  know  record  rajputana  name  region  george  thomas  1800  memoir  military  memory  john  keay  book  india  history  state  rajputana  coin  british  1829  john  briggs  translate  ferishtas  history  early  islamic  india  use  phrase  rajpoot  rajput  prince  rather  indian  prince    first  usage  word  rajasthan    
ANSWER:  1829; PREDICTION:  ---  ;;;  ---  ;;;  ---  ;;;  ---  ;;;  


In [141]:
predicted_word

0

In [381]:
ex = KerasBatchGenerator(valid_data,num_steps, 1, vocabulary, skip_step=0)
ex_data = next(ex.generate())

for i in range(100):
    ex_data = next(ex.generate())
    pred = model_glove_50d.predict(ex_data[0])
    x=0
    while (x==0 or x==2):
        x = np.argmax(pred[:,num_steps-1,:])
        pred[0 , num_steps-1, x] = 0
    print(i, " : ",  x)
    print(reversed_dictionary[x])

0  :  36596
use
1  :  36596
use
2  :  36596
use
3  :  36596
use
4  :  36596
use
5  :  36596
use
6  :  36596
use
7  :  36596
use
8  :  36596
use
9  :  36596
use
10  :  36596
use
11  :  36596
use
12  :  36596
use
13  :  36596
use


KeyboardInterrupt: 

In [340]:
np.argmax(out[]

(1, 313, 51236)

In [364]:
ex = KerasBatchGenerator(train_data[39:],num_steps, 1, vocabulary, skip_step=0)

ex_data = next(ex.generate())[0]
ans_index = np.argmax(pred[0, num_steps-1, :])

In [368]:
for i in range(20):
    seq=''
    inp=ex_data
    prediction_arr = model_glove_50d.predict(inp)
    answer_indices=[]
    for i in range(313):
        ans_index = np.argmax(pred[:, num_steps-i-1, :])
        answer_indices.append(ans_index)
    for j in range(len(answer_indices)):    
        ans = reversed_dictionary[answer_indices[j]]
        if (ans!='---' and ans!=''):
            seq+=ans
        
    print(seq)

one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one


In [ ]:
model_pos = 

In [401]:
ex = KerasBatchGenerator(train_data[50:],num_steps, 1, vocabulary, skip_step=0)
ex_data = next(ex.generate())

ex_data[0].shape
inp=ex_data[0]
out=ex_data[1]
ans=np.argmax(ex_data[1][:, num_steps-1, :])
prediction_arr = model_pos.predict(inp)

answer_list = []
pred = prediction_arr.copy()
max_answer_length = 10
while len(answer_list)<max_answer_length:
    ans_index = np.argmax(pred[0, num_steps-1, :])
    if (ans_index!=0 and ans_index!=2):
        print(ans_index)
        word = reversed_dictionary[ans_index]
        answer_list.append(word)
    pred[0 , num_steps-1, ans_index] = 0

seq=''
for x in inp[0,:]:
    if x!=0:
        seq += reversed_dictionary[x] + "  "
o = ans
seq += '\n' + "ANSWER:  " + reversed_dictionary[o] + "; "
seq += "PREDICTION:  " 
for j in range(max_answer_length):
    seq += "{}. ".format(j+1) + answer_list[j] + "  ;"
    
print(seq)



36596
43073
21681
13070
4854
34816
37926
21091
42559
16714
though  pesticide  regulation  differ  country  country  pesticide  product  use  trade  across  international  border  deal  inconsistency  regulation  among  country  delegate  conference  united  nation  food  agriculture  organization  adopt  international  code  conduct  distribution  use  pesticide  1985  create  voluntary  standard  pesticide  regulation  different  country  code  update  1998  2002  fao  claim  code  raise  awareness  pesticide  hazard  decrease  number  country  without  restriction  pesticide  use    recently  voluntary  standard  pesticide  regulation  update    
ANSWER:  2002; PREDICTION:  1. use  ;2. year  ;3. many  ;4. also  ;5. one  ;6. first  ;7. city  ;8. state  ;9. include  ;10. new  ;


In [145]:
ans1 = np.argmax(pred_copy[:, num_steps-1, :])
ans1

0

In [ ]:
36596
43073
21681
13070

In [128]:
dummy_iters = 40
example_training_generator = KerasBatchGenerator(train_data, num_steps, 1, vocabulary,
                                                     skip_step=num_steps+1)
print("Training data:")
for i in range(dummy_iters):
    dummy = next(example_training_generator.generate())
num_predict = 110
true_print_out = "Actual words: "
pred_print_out = "Predicted words: "
for i in range(num_predict):
    data = next(example_training_generator.generate())
    prediction = model_glove_50d.predict(data[0])
    predict_word = np.argmax(prediction[:, num_steps-1, :])
    predict_word = np.argsort(prediction[:, num_steps-1, :])[:3] # bear in mind this return the indices
    true_print_out += reversed_dictionary[train_data[num_steps + dummy_iters + i]] + " "
    pred_print_out += reversed_dictionary[predict_word] + " "
print(true_print_out)
print(pred_print_out)
print(prediction.shape)

Training data:


TypeError: unhashable type: 'numpy.ndarray'